In [2]:
from overcovregions.read_and_write import read_bedgraphlike_file
from overcovregions.coverage import find_overcov_regions

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
%load_ext autoreload
%autoreload 2

# Script usage

```
Usage : 
-------

python FindOverCovRegions.py [-h] -i INPUTFILEPATH -o OUTPUTDIR -p PREFIX -d DEPTH -m MINOVERCOVREGIONSIZE -s SLIDINGWINSIZE
```

The input file can be obtained using bedtools genomecov (https://bedtools.readthedocs.io/en/latest/content/tools/genomecov.html) with alignments bam files as follow:

```
bedtools genomecov -ibam <alignment.bam> [-d | -d split] ..
```
Each position must have a coverage depth value associated (per-base reports), even if it's 0 (see option -d or -d split in bedtools genomecov).

/!\ The BAM file must be sorted by position. Using samtools sort aln.bam aln.sorted will suffice.


The bedtools ouput file (input file of this script) must be formated as follow :

```
chr1   1    0   
chr1   2    5
chr1   3   10
...
```

where :
- the first column is the chr/scaffold/contig identifier
- the second column is the position
- the third column is the coverage depth.

(see '../test/begraph-like_sample.txt' for an example)

example on the toy dataset:

``` 
python FindOverCovRegions.py -i test/begraph-like_sample.txt -o test/ -prefix 'example_overcovregions' -d 11 -m 4 -s 3
```

# Use case 

## Data importation

In [46]:
### Open the bedgraph like file
#
df_data = read_bedgraphlike_file('../test/begraph-like_sample.txt')

### Sort the dataframe by 'contig' and 'position'
#
df_data = df_data.sort_values(by=['contig', 'position'])

In [47]:
### Count the number of positions per chr/scaffold/contigs
#
print('contig size (nb of position):\n')
df_data.contig.value_counts()

contig size (nb of position):



chr1    28
chr2    26
chr3     8
Name: contig, dtype: int64

## by hand

In [48]:
### Display the data. See the '../test/begraph-like_sample.txt' for a better
#   readability.
df_data

,contig,position,depth
0,chr1,1,4
1,chr1,2,11
2,chr1,3,12
3,chr1,4,15
4,chr1,5,18
...,...,...,...
57,chr3,4,10
58,chr3,5,9
59,chr3,6,10
60,chr3,7,8


Use case : we are looking for over-covered regions using the following parameters:
- i)   a sliding window with a width of 3 positions
- ii)  a coverage depth threshold of 11. The mean depth is computed on the sliding window and compared to this value
- iii) a minimal size of 4 for a an over-covered region to be retained.

The 3rd contig present no overcovered region. Only the following intervals satisfies the two first parameters :
```
contig start end length

chr1   2    10    9
chr1  14    21    7
chr1  24    29    6
chr2   1    11   11
chr2  14    17    4
chr2  21    23    3
```

But the last over-covered region is too short (len : 3 < min. over-covered region's length (4), thus this region is not kept. 

The over-covered regions are :

```
contig start end 

chr1   2    10   
chr1  14    21   
chr1  24    29   
chr2   1    11  
chr2  14    17   
```

## Computation

In [52]:
### set the parameters values
#
sliding_win_len = 3
depth_thresh = 11
region_min_len = 4


### iterate through 'contigs' and look for over-covered regions
#
lst_df_overcov = []
for contig in df_data.contig.unique():
    df_contig = df_data[df_data.contig==contig]
    lst_df_overcov.append(
        find_overcov_regions(df_data[df_data.contig==contig],
                             sliding_win_len=sliding_win_len,
                             depth_thresh=depth_thresh,
                            )
    )
    
### create a pd.DataFrame containing the results for every contig
#
df_overcov_regions = pd.concat(lst_df_overcov)
df_overcov_regions

,contig,start,end,length,median_depth,mean_depth,std_depth
0,chr1,2,10,9,15.0,13.555556,4.786813
1,chr1,14,21,7,12.0,12.428571,4.237828
2,chr1,24,29,6,12.5,13.666667,3.197221
0,chr2,1,11,11,18.0,14.545455,4.831029
1,chr2,14,17,4,11.5,11.500000,1.118034
2,chr2,21,23,3,11.0,11.000000,0.816497


In [53]:
### Remove regions that are too short
#
df_overcov_regions = df_overcov_regions[df_overcov_regions.length >= region_min_len]
df_overcov_regions

,contig,start,end,length,median_depth,mean_depth,std_depth
0,chr1,2,10,9,15.0,13.555556,4.786813
1,chr1,14,21,7,12.0,12.428571,4.237828
2,chr1,24,29,6,12.5,13.666667,3.197221
0,chr2,1,11,11,18.0,14.545455,4.831029
1,chr2,14,17,4,11.5,11.500000,1.118034
